In [5]:
import pickle
import pickle as pkl
import json
import sqlite3 as sql
from pathlib import Path
import pandas as pd
from rdkit.Chem import Draw
from rdkit import Chem
from PIL import Image

In [6]:
DATA_DIR = Path('..', 'data').resolve()
OUTPUT_DIR = DATA_DIR / 'outputs'
INFO_PATH = OUTPUT_DIR / 'library_constituents_dataframe.pkl'
MAPPING_PATH = OUTPUT_DIR / 'compound_mapping.txt'
EXP_PLAN_PATH = OUTPUT_DIR / 'synthesis_plan.json'
DB_PATH = DATA_DIR / 'db' / '50k_project.db'
STATIC_DIR = DATA_DIR / 'db' / 'static'

In [7]:
with open(INFO_PATH, 'rb') as file:
    df = pd.read_pickle(file)

with open(MAPPING_PATH, 'r') as file:
    compound_mapping = {}
    for line in file.readlines():
        compound_mapping[line.split()[0]] = line.split()[1]

with open(EXP_PLAN_PATH, 'r')as file:
    synthesis_plan = json.load(file)

con = sql.connect(DB_PATH)

In [8]:
# lets try with I1
short = 'I1'

In [10]:
for short in compound_mapping.keys():
    long = compound_mapping[short]
    mol = df.loc[df['Compound Name'] == long, 'mol'].item()
    smiles = Chem.MolToSmiles(mol)  # we regenerate SMILES instead of using the value from df to have a canonical representation
    image = Draw.MolToImage(mol)  # TODO change this into the better drawing method
    category = df.loc[df['Compound Name'] == long,'Category'].item()
    # save image to static dir and keep imagepath to add to db
    image_path = STATIC_DIR / 'image' / f'{short}.png'
    image.save(image_path)
    # write this to the db
    cur = con.cursor()
    cur.execute('INSERT INTO main.buildingblocks(long, short, SMILES, image, category, boc, cbz, tbu, tms) VALUES(?,?,?,?,?,?,?,?,?);', (long, short, smiles, str(image_path.resolve()), category, boc, cbz, tbu, tms))

In [11]:
con.commit()

In [22]:
con.close()